In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from collections import defaultdict
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, make_scorer
from scipy.special import expit
from sklearn.model_selection import GridSearchCV, validation_curve, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.tree import plot_tree
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [9]:
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor

In [22]:
from lightgbm import LGBMClassifier

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [17]:
from xgboost import XGBClassifier

In [5]:
df_track  = pd.read_csv('df_track_no_duplicates.csv', skipinitialspace=True)

In [6]:
value_map = {'reggae': 'latin', 'reggaeton': 'latin', 'latino': 'latin','honky-tonk':'latin',
             'electro': 'edm', 'house': 'edm','idm':'edm',
             'indie-pop': 'indie',
             'minimal-techno': 'techno', 'electronic':'techno','chicago-house':'techno','detroit-techno':'techno','deep-house':'techno','industrial':'techno','hardstyle':'techno',
             'dub': 'dubstep',
             'progressive-house': 'trance',
             'samba': 'brazil', 'pagode': 'brazil', 'forro': 'brazil', 'sertanejo': 'brazil','mpb':'brazil',
             'rockabilly': 'rock','rock-n-roll':'rock',
             'world-music': 'ambient',
             'cantopop': 'mandopop',
             'synth-pop':'pop','pop-film':'pop',
             'metalcore': 'metal','death-metal':'metal','black-metal':'metal','eavy-metal':'metal','grindcore':'metal','grunge':'metal',
             'punk-rock': 'punk',
             'alt-rock': 'rock', 'alternative': 'rock','power-pop':'rock','psych-rock':'rock','hardcore':'rock','hard-rock':'rock',
             'j-pop':'japan','j-rock':'japan','anime':'japan','j-dance':'japan','j-idol':'japan',
             'classical':'opera',
             'sad':'emo','chill':'emo','goth':'emo',
             'dancehall':'dance','disco':'dance','party':'dance',
             'disney':'kids','children':'kids',
             'breakbeat':'hip-hop','drum-and-bass':'hip-hop','r-n-b':'hip-hop','trip-hop':'hip-hop','club':'hip-hop','ska':'hip-hop',
             'acoustic':'guitar',
             'soul':'blues',
             'salsa':'spanish','tango':'spanish',
             } 

df_track['genre_cluster'] = df_track['genre'].replace(value_map)

In [7]:
columns = ['duration_sec', 'popularity', 'danceability', 'energy', 'loudness', 'speechiness',
           'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'fade_out_%', 
           'n_beats']
df_sub = df_track[columns]
X = df_sub.values
#y = df_track['genre_cluster'].values
y = np.array(df_track['genre_cluster'])

X.shape, y.shape

((83612, 13), (83612,))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((58528, 13), (25084, 13), (58528,), (25084,))

# GB 

In [30]:
clf = GradientBoostingClassifier(n_estimators=100, 
                                 learning_rate=1.0, max_depth=3, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

TypeError: GradientBoostingClassifier.__init__() got an unexpected keyword argument 'n_jobs'

# XGBoost

In [23]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Codifica le etichette delle classi in valori interi
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Salva la mappatura tra numeri e nomi delle classi
class_mapping = {i: cls for i, cls in enumerate(label_encoder.classes_)}

# Definisci e addestra il classificatore XGBoost
clf = XGBClassifier(objective='multi:softmax', 
                    num_class=len(label_encoder.classes_),  
                    max_depth=6,
                    learning_rate=0.3,
                    gamma=0.0,
                    reg_lambda=1,
                    tree_method='exact', 
                    use_label_encoder=False,
                    random_state=42)
clf.fit(X_train, y_train_encoded)

# Effettua le predizioni
y_pred_encoded = clf.predict(X_test)

# Decodifica le predizioni nelle etichette originali delle classi
y_pred = [class_mapping[pred] for pred in y_pred_encoded]

# Calcola le metriche di valutazione
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))


#learning rate= 1.0 acc: 0.27
#learning rate= 0.3 acc: 0.42

Accuracy 0.4153643756976559
F1-score [0.28512397 0.38577778 0.44112769 0.10428305 0.57675753 0.0754717
 0.85357143 0.24444444 0.32587413 0.33449477 0.3081761  0.31567568
 0.05524862 0.18266979 0.32209738 0.16348774 0.170347   0.28571429
 0.13720317 0.28488932 0.43146067 0.33687943 0.36136553 0.16509434
 0.13265306 0.55555556 0.39784533 0.21276596 0.33557047 0.47956731
 0.52293578 0.19409283 0.41885626 0.59776347 0.48880597 0.49559471
 0.40978593 0.27444254 0.1443299  0.37141393 0.58105939 0.31456311
 0.79928952 0.10218978 0.46931818 0.73783359 0.04575163 0.53825011
 0.47692308 0.19066148]
              precision    recall  f1-score   support

    afrobeat       0.36      0.24      0.29       292
     ambient       0.38      0.39      0.39       554
   bluegrass       0.43      0.45      0.44       293
       blues       0.15      0.08      0.10       347
      brazil       0.49      0.69      0.58      1454
     british       0.18      0.05      0.08       167
      comedy       0.91  

# Lightgbm

In [25]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

clf = LGBMClassifier(boosting_type='gbdt',
                     max_depth=-1,
                     num_class=len(label_encoder.classes_),  
                     num_leaves=31,
                     n_estimators=100,
                     subsample_for_bin=200000,
                     objective='multiclass',
                     reg_alpha=0.0,
                     reg_lambda=0.0,
                     random_state=42
                   )
clf.fit(X_train, y_train_encoded)
y_pred_encoded = clf.predict(X_test)

# Decodifica le predizioni nelle etichette originali delle classi
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Calcola le metriche di valutazione
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001909 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3153
[LightGBM] [Info] Number of data points in the train set: 58528, number of used features: 13
[LightGBM] [Info] Start training from score -4.453698
[LightGBM] [Info] Start training from score -3.811767
[LightGBM] [Info] Start training from score -4.447842
[LightGBM] [Info] Start training from score -4.280226
[LightGBM] [Info] Start training from score -2.848380
[LightGBM] [Info] Start training from score -5.008553
[LightGBM] [Info] Start training from score -4.434789
[LightGBM] [Info] Start training from score -5.061058
[LightGBM] [Info] Start training from score -3.600127
[LightGBM] [Info] Start training from score -4.369260
[LightGBM] [Info] Start training from score -3.669388
[LightGBM] [Info] Start training from score -3.298934
[LightGBM] [Info] Start training from score -5.096728
[LightGBM

# CatBoost

In [27]:
from catboost import CatBoostClassifier

In [28]:
clf = CatBoostClassifier()
clf.fit(X_train, y_train)

Learning rate set to 0.097378
0:	learn: 3.6037377	total: 502ms	remaining: 8m 21s
1:	learn: 3.4585280	total: 854ms	remaining: 7m 6s
2:	learn: 3.3387536	total: 1.18s	remaining: 6m 32s
3:	learn: 3.2296855	total: 1.51s	remaining: 6m 15s
4:	learn: 3.1583331	total: 1.81s	remaining: 6m 1s
5:	learn: 3.0985019	total: 2.13s	remaining: 5m 52s
6:	learn: 3.0437028	total: 2.44s	remaining: 5m 45s
7:	learn: 2.9948873	total: 2.73s	remaining: 5m 39s
8:	learn: 2.9504535	total: 3.08s	remaining: 5m 38s
9:	learn: 2.9055063	total: 3.39s	remaining: 5m 35s
10:	learn: 2.8662118	total: 3.74s	remaining: 5m 36s
11:	learn: 2.8275237	total: 4.05s	remaining: 5m 33s
12:	learn: 2.7903203	total: 4.36s	remaining: 5m 31s
13:	learn: 2.7619835	total: 4.66s	remaining: 5m 28s
14:	learn: 2.7357098	total: 4.97s	remaining: 5m 26s
15:	learn: 2.7086241	total: 5.28s	remaining: 5m 24s
16:	learn: 2.6922038	total: 5.58s	remaining: 5m 22s
17:	learn: 2.6747948	total: 5.88s	remaining: 5m 20s
18:	learn: 2.6512993	total: 6.19s	remaining: 5

In [29]:
y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.44350980704831766
F1-score [0.31075697 0.4294699  0.46405229 0.10984848 0.5867052  0.10362694
 0.83972125 0.27681661 0.38001407 0.3261944  0.3490566  0.33643836
 0.08421053 0.21266968 0.35789474 0.15053763 0.14864865 0.3373494
 0.14948454 0.30959752 0.50772627 0.36998255 0.3901717  0.1749409
 0.17322835 0.5968254  0.43528505 0.25641026 0.38477801 0.5244713
 0.55341506 0.19126819 0.44189602 0.61893451 0.51724138 0.54189336
 0.49132948 0.3238255  0.14814815 0.39425051 0.61783439 0.33013436
 0.81283422 0.10646388 0.50556793 0.73185185 0.04026846 0.57267911
 0.48298573 0.20233463]
              precision    recall  f1-score   support

    afrobeat       0.37      0.27      0.31       292
     ambient       0.43      0.43      0.43       554
   bluegrass       0.45      0.48      0.46       293
       blues       0.16      0.08      0.11       347
      brazil       0.51      0.70      0.59      1454
     british       0.38      0.06      0.10       167
      comedy       0.87   